# Final Project

## Phase 1: Data cleaning

In [1]:
library(stringr)
library(dplyr)


Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union



In [2]:
df <- jsonlite::fromJSON('fall2019-survey-M05.json') #importing the json file into a dataframe

In [3]:
head(df)

Destination.City,Origin.City,Airline.Status,Age,Gender,Price.Sensitivity,Year.of.First.Flight,Flights.Per.Year,Loyalty,Type.of.Travel,...,Arrival.Delay.in.Minutes,Flight.cancelled,Flight.time.in.minutes,Flight.Distance,Likelihood.to.recommend,olong,olat,dlong,dlat,freeText
"Phoenix, AZ","Minneapolis, MN",Blue,30,Female,3,2012,6,0.0769,Business travel,...,0,No,165,1276,7,-93.2798,44.9782,-111.9700,33.4941,NA
"Salt Lake City, UT","Sacramento, CA",Silver,57,Female,1,2007,27,-0.6875,Personal Travel,...,2,No,85,532,7,-121.3247,38.6386,-111.9295,40.6408,NA
"Atlanta, GA","Las Vegas, NV",Silver,75,Female,3,2009,35,-0.8421,Personal Travel,...,0,No,191,1747,8,-115.1672,36.1349,-84.3319,33.8241,NA
"San Francisco, CA","New Orleans, LA",Blue,55,Female,1,2005,13,-0.6250,Business travel,...,0,No,257,1911,9,-90.1437,29.9547,-122.1585,37.6712,NA
"Orlando, FL","Atlanta, GA",Blue,80,Male,0,2003,77,-0.9494,Personal Travel,...,0,No,54,404,4,-84.3319,33.8241,-81.4112,28.5844,NA
"Chicago, IL","Cleveland, OH",Blue,48,Female,2,2012,52,-0.8909,Personal Travel,...,13,No,72,315,4,-81.6054,41.4436,-87.8954,41.8271,NA


In [4]:
str(df) # structure of the raw dataframe

'data.frame':	10282 obs. of  32 variables:
 $ Destination.City              : chr  "Phoenix, AZ" "Salt Lake City, UT" "Atlanta, GA" "San Francisco, CA" ...
 $ Origin.City                   : chr  "Minneapolis, MN" "Sacramento, CA" "Las Vegas, NV" "New Orleans, LA" ...
 $ Airline.Status                : chr  "Blue" "Silver" "Silver" "Blue" ...
 $ Age                           : int  30 57 75 55 80 48 68 31 17 41 ...
 $ Gender                        : chr  "Female" "Female" "Female" "Female" ...
 $ Price.Sensitivity             : int  3 1 3 1 0 2 1 1 2 1 ...
 $ Year.of.First.Flight          : int  2012 2007 2009 2005 2003 2012 2006 2004 2006 2009 ...
 $ Flights.Per.Year              : int  6 27 35 13 77 52 33 15 8 3 ...
 $ Loyalty                       : num  0.0769 -0.6875 -0.8421 -0.625 -0.9494 ...
 $ Type.of.Travel                : chr  "Business travel" "Personal Travel" "Personal Travel" "Business travel" ...
 $ Total.Freq.Flyer.Accts        : int  0 0 0 0 0 0 0 2 1 0 ...
 $ Shoppin

In [5]:
#Remove the State names from the Destination.City and Origin.City as States columns are already present
df$Destination.City <- str_remove(df$Destination.City,", [:alpha:]+")
df$Origin.City <- str_remove(df$Origin.City,", [:alpha:]+")
# Changing the Flight.date column's data type to date
df$Flight.date <- as.Date(df$Flight.date,format="%m/%d/%y")
# Shrinking the Gender and Flight.cancelled columns 
df$Gender <- str_sub(df$Gender,1,1)
df$Flight.cancelled <- str_sub(df$Flight.cancelled ,1,1)

In [6]:
colSums(is.na(df)) # counting the number of NA values in each columns 

Destination.City                    Origin.City 
                             0                              0 
                Airline.Status                            Age 
                             0                              0 
                        Gender              Price.Sensitivity 
                             0                              0 
          Year.of.First.Flight               Flights.Per.Year 
                             0                              0 
                       Loyalty                 Type.of.Travel 
                             0                              0 
        Total.Freq.Flyer.Accts     Shopping.Amount.at.Airport 
                             0                              0 
Eating.and.Drinking.at.Airport                          Class 
                             0                              0 
                  Day.of.Month                    Flight.date 
                             0                              0 
                  Partner.Code                   Partner.Name 
                             0                              0 
                  Origin.State              Destination.State 
                             0                              0 
      Scheduled.Departure.Hour     Departure.Delay.in.Minutes 
                             0                            212 
      Arrival.Delay.in.Minutes               Flight.cancelled 
                           248                              0 
        Flight.time.in.minutes                Flight.Distance 
                           248                              0 
       Likelihood.to.recommend                          olong 
                             1                              0 
                          olat                          dlong 
                             0                              0 
                          dlat                       freeText 
                             0                          10000

In [7]:
# removing the one row with Likelihood.to.recommend as NA
df <- df[-which(is.na(df$Likelihood.to.recommend)),] 

In [8]:
df$freeText <- NULL #droping the last column with NA values

In [9]:
# filling the NA values in the Flight.time.in.minutes  column.
# this is done as follow
# 1) for each row where the Flight.time.in.minutes is null, destination and origin city are taken.
# 2) filter out the list of rows where the destination and origin city are the same from the dataframe.
# 3) take the Flight.time.in.minutes values from the list and find the mean and median
# 4) replace the NA with the min of mean and median of the list

l <- which(is.na(df$Flight.time.in.minutes))

for (i in l){
    minidf <- df %>% 
                filter(Destination.City == df[i,]$Destination.City) %>%
                filter(Origin.City == df[i,]$Origin.City) %>%
                pull(Flight.time.in.minutes)
    
    df[i,]$Flight.time.in.minutes <- (min(mean(minidf,na.rm=TRUE),median(minidf,na.rm=TRUE)))
}


In [10]:
# filling the NA values in the Arrival.Delay.in.Minutes column.
# this is done as follow
# 1) for each row where the Arrival.Delay.in.Minutes is null, destination city and flight partner are taken.
# 2) filter out the list of rows where the destination city and flight partner are the same from the dataframe.
# 3) take the Arrival.Delay.in.Minutes values from the list and find the median
# 4) replace the NA with the median of the list


l <- which(is.na(df$Arrival.Delay.in.Minutes))

for (i in l){
        mod <- df %>% 
                filter(Destination.City == df[i,]$Destination.City) %>%
                filter(Partner.Name == df[i,]$Partner.Name) %>%
                pull(Arrival.Delay.in.Minutes) 
    
    df[i,]$Arrival.Delay.in.Minutes <- median(minidf,na.rm=TRUE)
}

In [11]:
# filling the NA values in the Departure.Delay.in.Minutes column.
# this is done as follow
# 1) for each row where the Departure.Delay.in.Minutes is null, origin city and flight partner are taken.
# 2) filter out the list of rows where the origin city and flight partner are the same from the dataframe.
# 3) take the Arrival.Delay.in.Minutes values from the list and find the median
# 4) replace the NA with the median of the list


l <- which(is.na(df$Departure.Delay.in.Minutes))

for (i in l){
        mod <- df %>% 
                filter(Origin.City == df[i,]$Origin.City) %>%
                filter(Partner.Name == df[i,]$Partner.Name) %>%
                pull(Departure.Delay.in.Minutes) 
    
    df[i,]$Departure.Delay.in.Minutes <- median(minidf,na.rm=TRUE)
}

In [12]:
# removing the 13 rows with Flight.time.in.minutes as NA
df <- df[-which(is.na(df$Flight.time.in.minutes)),]

In [13]:
head(df)

Destination.City,Origin.City,Airline.Status,Age,Gender,Price.Sensitivity,Year.of.First.Flight,Flights.Per.Year,Loyalty,Type.of.Travel,...,Departure.Delay.in.Minutes,Arrival.Delay.in.Minutes,Flight.cancelled,Flight.time.in.minutes,Flight.Distance,Likelihood.to.recommend,olong,olat,dlong,dlat
Phoenix,Minneapolis,Blue,30,F,3,2012,6,0.0769,Business travel,...,0,0,N,165,1276,7,-93.2798,44.9782,-111.9700,33.4941
Salt Lake City,Sacramento,Silver,57,F,1,2007,27,-0.6875,Personal Travel,...,0,2,N,85,532,7,-121.3247,38.6386,-111.9295,40.6408
Atlanta,Las Vegas,Silver,75,F,3,2009,35,-0.8421,Personal Travel,...,5,0,N,191,1747,8,-115.1672,36.1349,-84.3319,33.8241
San Francisco,New Orleans,Blue,55,F,1,2005,13,-0.6250,Business travel,...,0,0,N,257,1911,9,-90.1437,29.9547,-122.1585,37.6712
Orlando,Atlanta,Blue,80,M,0,2003,77,-0.9494,Personal Travel,...,0,0,N,54,404,4,-84.3319,33.8241,-81.4112,28.5844
Chicago,Cleveland,Blue,48,F,2,2012,52,-0.8909,Personal Travel,...,0,13,N,72,315,4,-81.6054,41.4436,-87.8954,41.8271


In [14]:
colSums(is.na(df))

Destination.City                    Origin.City 
                             0                              0 
                Airline.Status                            Age 
                             0                              0 
                        Gender              Price.Sensitivity 
                             0                              0 
          Year.of.First.Flight               Flights.Per.Year 
                             0                              0 
                       Loyalty                 Type.of.Travel 
                             0                              0 
        Total.Freq.Flyer.Accts     Shopping.Amount.at.Airport 
                             0                              0 
Eating.and.Drinking.at.Airport                          Class 
                             0                              0 
                  Day.of.Month                    Flight.date 
                             0                              0 
                  Partner.Code                   Partner.Name 
                             0                              0 
                  Origin.State              Destination.State 
                             0                              0 
      Scheduled.Departure.Hour     Departure.Delay.in.Minutes 
                             0                              0 
      Arrival.Delay.in.Minutes               Flight.cancelled 
                             0                              0 
        Flight.time.in.minutes                Flight.Distance 
                             0                              0 
       Likelihood.to.recommend                          olong 
                             0                              0 
                          olat                          dlong 
                             0                              0 
                          dlat 
                             0

In [15]:
df$Airline.Status <- as.factor(as.character(df$Airline.Status))
df$Gender<- as.factor(as.character(df$Gender))
df$Type.of.Travel <- as.factor(as.character(df$Type.of.Travel))
df$Class <- as.factor(as.character(df$Class))
df$Partner.Name <- as.factor(as.character(df$Partner.Name))
df$Partner.Code <- as.factor(as.character(df$Partner.Code))
df$Origin.State <- as.factor(as.character(df$Origin.State))
df$Destination.State <- as.factor(as.character(df$Destination.State))
df$Origin.City <- as.factor(as.character(df$Origin.City))
df$Destination.City <- as.factor(as.character(df$Destination.City))
df$Flight.cancelled  <- as.factor(as.character(df$Flight.cancelled))
df$Year.of.First.Flight  <- as.factor(as.character(df$Year.of.First.Flight))

In [16]:
str(df)

'data.frame':	10268 obs. of  31 variables:
 $ Destination.City              : Factor w/ 195 levels "Aberdeen","Aguadilla",..: 139 154 13 157 133 38 83 119 83 50 ...
 $ Origin.City                   : Factor w/ 192 levels "Aberdeen","Aguadilla",..: 119 153 104 128 13 41 35 187 25 157 ...
 $ Airline.Status                : Factor w/ 4 levels "Blue","Gold",..: 1 4 4 1 1 1 1 1 1 1 ...
 $ Age                           : int  30 57 75 55 80 48 68 31 17 41 ...
 $ Gender                        : Factor w/ 2 levels "F","M": 1 1 1 1 2 1 2 1 1 2 ...
 $ Price.Sensitivity             : int  3 1 3 1 0 2 1 1 2 1 ...
 $ Year.of.First.Flight          : Factor w/ 10 levels "2003","2004",..: 10 5 7 3 1 10 4 2 4 7 ...
 $ Flights.Per.Year              : int  6 27 35 13 77 52 33 15 8 3 ...
 $ Loyalty                       : num  0.0769 -0.6875 -0.8421 -0.625 -0.9494 ...
 $ Type.of.Travel                : Factor w/ 3 levels "Business travel",..: 1 3 3 1 3 3 3 2 1 1 ...
 $ Total.Freq.Flyer.Accts        : int 

In [17]:
saveRDS(df, file='data.rds')